In [ ]:
import sys
import os 
import nest_asyncio

# Sanity check
print(sys.executable)
nest_asyncio.apply()

os.environ["OPENAI_API_KEY"] = "sk-"
#os.environ[
#    "AZURE_OPENAI_ENDPOINT"
#] = "https://<your-resource-name>.openai.azure.com/"
#os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"

In [ ]:
import os
from pydantic import BaseModel, Field
from llama_index.core.workflow import (
    Workflow,
    step,
    Event,
    Context,
    StartEvent,
    StopEvent
)
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine


In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O './paul_graham_essay.txt'

In [ ]:
llm = OpenAI(temperature=0.2, model="gpt-4o-mini")

## 1 - RAG 

Using the BM25 retriever system 

In [ ]:

documents = SimpleDirectoryReader(
    input_files=["./paul_graham_essay.txt"],
).load_data()
splitter = SentenceSplitter(chunk_size=256)
nodes = splitter.get_nodes_from_documents(documents)
retriever_top_5 = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=5,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

In [ ]:
rez = retriever_top_5.retrieve("computer")

print(rez[0])

In [ ]:
print(rez[1].text)

In [ ]:
# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever_top_5,
    response_synthesizer=response_synthesizer,
)
response = query_engine.query("Who is Paul Graham.")

In [ ]:
response.response

## 2 - Exercise : 

Combine it with Workflows

Create a workflow that : 
- Search for the best quote about the user query
- Make a rap about it




In [ ]:

class ContextualGrahamRapWorkflow(Workflow):
    
    @step
    def do(self, ev: StartEvent) -> StopEvent:
        return StopEvent()



In [ ]:
w = ContextualGrahamRapWorkflow()

r = await w.run()

## 3 - Exercise (if time permits) 

Combine it with a reranker

Create a workflow that : 
- Search for the best quote about the user query
- Rerank it
- Make a rap about it
